In [0]:
import pandas as pd
import time

from google.cloud import bigquery


class BigQueryHelper(object):
    """
    Helper class to simplify common BigQuery tasks like executing queries,
    showing table schemas, etc without worrying about table or dataset pointers.

    See the BigQuery docs for details of the steps this class lets you skip:
    https://googlecloudplatform.github.io/google-cloud-python/latest/bigquery/reference.html
    """

    def __init__(self, active_project, dataset_name, max_wait_seconds=180):
        self.project_name = active_project
        self.dataset_name = dataset_name
        self.max_wait_seconds = max_wait_seconds
        self.client = bigquery.Client()
        self.__dataset_ref = self.client.dataset(self.dataset_name, project=self.project_name)
        self.dataset = None
        self.tables = dict()  # {table name (str): table object}
        self.__table_refs = dict()  # {table name (str): table reference}
        self.total_gb_used_net_cache = 0
        self.BYTES_PER_GB = 2**30

    def __fetch_dataset(self):
        """
        Lazy loading of dataset. For example,
        if the user only calls `self.query_to_pandas` then the
        dataset never has to be fetched.
        """
        if self.dataset is None:
            self.dataset = self.client.get_dataset(self.__dataset_ref)

    def __fetch_table(self, table_name):
        """
        Lazy loading of table
        """
        self.__fetch_dataset()
        if table_name not in self.__table_refs:
            self.__table_refs[table_name] = self.dataset.table(table_name)
        if table_name not in self.tables:
            self.tables[table_name] = self.client.get_table(self.__table_refs[table_name])

    def __handle_record_field(self, row, schema_details, top_level_name=''):
        """
        Unpack a single row, including any nested fields.
        """
        name = row['name']
        if top_level_name != '':
            name = top_level_name + '.' + name
        schema_details.append([{
            'name': name,
            'type': row['type'],
            'mode': row['mode'],
            'fields': pd.np.nan,
            'description': row['description']
                               }])
        # float check is to dodge row['fields'] == np.nan
        if type(row.get('fields', 0.0)) == float:
            return None
        for entry in row['fields']:
            self.__handle_record_field(entry, schema_details, name)

    def __unpack_all_schema_fields(self, schema):
        """
        Unrolls nested schemas. Returns dataframe with one row per field,
        and the field names in the format accepted by the API.
        Results will look similar to the website schema, such as:
            https://bigquery.cloud.google.com/table/bigquery-public-data:github_repos.commits?pli=1

        Args:
            schema: DataFrame derived from api repr of raw table.schema
        Returns:
            Dataframe of the unrolled schema.
        """
        schema_details = []
        schema.apply(lambda row:
            self.__handle_record_field(row, schema_details), axis=1)
        result = pd.concat([pd.DataFrame.from_dict(x) for x in schema_details])
        result.reset_index(drop=True, inplace=True)
        del result['fields']
        return result

    def table_schema(self, table_name):
        """
        Get the schema for a specific table from a dataset.
        Unrolls nested field names into the format that can be copied
        directly into queries. For example, for the `github.commits` table,
        the this will return `committer.name`.

        This is a very different return signature than BigQuery's table.schema.
        """
        self.__fetch_table(table_name)
        raw_schema = self.tables[table_name].schema
        schema = pd.DataFrame.from_dict([x.to_api_repr() for x in raw_schema])
        # the api_repr only has the fields column for tables with nested data
        if 'fields' in schema.columns:
            schema = self.__unpack_all_schema_fields(schema)
        # Set the column order
        schema = schema[['name', 'type', 'mode', 'description']]
        return schema

    def list_tables(self):
        """
        List the names of the tables in a dataset
        """
        self.__fetch_dataset()
        return([x.table_id for x in self.client.list_tables(self.dataset)])

    def estimate_query_size(self, query):
        """
        Estimate gigabytes scanned by query.
        Does not consider if there is a cached query table.
        See https://cloud.google.com/bigquery/docs/reference/rest/v2/jobs#configuration.dryRun
        """

        my_job_config = bigquery.job.QueryJobConfig()

        my_job_config.use_legacy_sql = True

        my_job_config.dry_run = True
        my_job = self.client.query(query, job_config=my_job_config)
        return my_job.total_bytes_processed / self.BYTES_PER_GB

    def query_to_pandas(self, query):
        """
        Execute a SQL query & return a pandas dataframe
        """
        my_job = self.client.query(query)
        start_time = time.time()
        while not my_job.done():
            if (time.time() - start_time) > self.max_wait_seconds:
                print("Max wait time elapsed, query cancelled.")
                self.client.cancel_job(my_job.job_id)
                return None
            time.sleep(0.1)
        # Queries that hit errors will return an exception type.
        # Those exceptions don't get raised until we call my_job.to_dataframe()
        # In that case, my_job.total_bytes_billed can be called but is None
        if my_job.total_bytes_billed:
            self.total_gb_used_net_cache += my_job.total_bytes_billed / self.BYTES_PER_GB
        return my_job.to_dataframe()

    def query_to_pandas_safe(self, query, max_gb_scanned=1):
        """
        Execute a query, but only if the query would scan less than `max_gb_scanned` of data.
        """
        query_size = self.estimate_query_size(query)
        if query_size <= max_gb_scanned:
            return self.query_to_pandas(query)
        msg = "Query cancelled; estimated size of {0} exceeds limit of {1} GB"
        print(msg.format(query_size, max_gb_scanned))

    def head(self, table_name, num_rows=5, start_index=None, selected_columns=None):
        """
        Get the first n rows of a table as a DataFrame.
        Does not perform a full table scan; should use a trivial amount of data as long as n is small.
        """
        self.__fetch_table(table_name)
        active_table = self.tables[table_name]
        schema_subset = None
        if selected_columns:
            schema_subset = [col for col in active_table.schema if col.name in selected_columns]
        results = self.client.list_rows(active_table, selected_fields=schema_subset,
            max_results=num_rows, start_index=start_index)
        results = [x for x in results]
        return pd.DataFrame(
            data=[list(x.values()) for x in results], columns=list(results[0].keys()))

In [3]:
from google.colab import auth
auth.authenticate_user()
print('Authenticated')

Authenticated


In [0]:
%load_ext google.colab.data_table

In [5]:
project_id = 'cool-eye-258712'
%env GCLOUD_PROJECT = cool-eye-258712

env: GCLOUD_PROJECT=cool-eye-258712


In [0]:

QUERY = """
        SELECT
  COUNT(1) AS number_of_non_variants
FROM
  `bigquery-public-data.human_genome_variants.platinum_genomes_deepvariant_variants_20180823` v, v.call call
WHERE
  NOT EXISTS (SELECT 1
                FROM UNNEST(v.alternate_bases) AS alt
              WHERE
                alt.alt NOT IN ("<NON_REF>", "<*>"))
        """

bq_assistant = BigQueryHelper("bigquery-public-data", "human_genome_variants")
bq_assistant.estimate_query_size(QUERY)



0.5340448450297117

In [0]:
%%bigquery --project cool-eye-258712

# SELECT COUNT(1) AS number_of_rows
# FROM `bigquery-public-data.human_genome_variants.platinum_genomes_deepvariant_variants_20180823`

# SELECT
#   SUM(ARRAY_LENGTH(call)) AS number_of_calls
# FROM
#   `bigquery-public-data.human_genome_variants.platinum_genomes_deepvariant_variants_20180823`

# SELECT COUNT(call) AS number_of_calls
# FROM `bigquery-public-data.human_genome_variants.platinum_genomes_deepvariant_variants_20180823` v, v.call

# SELECT
#   COUNT(call.name) AS number_of_calls
# FROM
#   `bigquery-public-data.human_genome_variants.platinum_genomes_deepvariant_variants_20180823` v, v.call call

# SELECT
#   COUNT(1) AS number_of_real_variants
# FROM
#   `bigquery-public-data.human_genome_variants.platinum_genomes_deepvariant_variants_20180823` v, v.call call
# WHERE
#   EXISTS (SELECT 1
#             FROM UNNEST(v.alternate_bases) AS alt
#           WHERE
#             alt.alt NOT IN ("<NON_REF>", "<*>"))

SELECT
  COUNT(1) AS number_of_non_variants
FROM
  `bigquery-public-data.human_genome_variants.platinum_genomes_deepvariant_variants_20180823` v, v.call call
WHERE
  NOT EXISTS (SELECT 1
                FROM UNNEST(v.alternate_bases) AS alt
              WHERE
                alt.alt NOT IN ("<NON_REF>", "<*>"))

,number_of_non_variants
0,143555264


In [0]:
QUERY = """
        SELECT message
        FROM `bigquery-public-data.github_repos.commits`
        WHERE LENGTH(message) > 6 AND LENGTH(message) <= 20
        LIMIT 2000
        """
bq_assistant = BigQueryHelper("bigquery-public-data", "github_repos")
bq_assistant.estimate_query_size(QUERY)

20.04199816659093

In [0]:
%%bigquery --project cool-eye-258712
SELECT
  call.name AS call_name,
  COUNT(call.name) AS call_count_for_call_set
FROM
  `bigquery-public-data.human_genome_variants.platinum_genomes_deepvariant_variants_20180823` v, v.call
GROUP BY
  call_name
ORDER BY
  call_name

,call_name,call_count_for_call_set
0,NA12877,31592135
1,NA12878,28012646
2,NA12889,31028550
3,NA12890,30636087
4,NA12891,33487348
5,NA12892,27347886


In [0]:
%%bigquery --project cool-eye-258712
SELECT
  call.name AS call_name,
  COUNT(call.name) AS call_count_for_call_set
FROM
  `bigquery-public-data.human_genome_variants.platinum_genomes_deepvariant_variants_20180823` v, v.call
WHERE
  EXISTS (SELECT 1
            FROM UNNEST(v.alternate_bases) AS alt
          WHERE
            alt.alt NOT IN ("<NON_REF>", "<*>"))
GROUP BY
  call_name
ORDER BY
  call_name

,call_name,call_count_for_call_set
0,NA12877,6284275
1,NA12878,6397315
2,NA12889,6407532
3,NA12890,6448600
4,NA12891,6516669
5,NA12892,6494997


In [0]:
%%bigquery --project cool-eye-258712
SELECT
  call.name AS call_name,
  COUNT(call.name) AS call_count_for_call_set
FROM
  `bigquery-public-data.human_genome_variants.platinum_genomes_deepvariant_variants_20180823` v, v.call
WHERE
  EXISTS (SELECT 1 FROM UNNEST(call.genotype) AS gt WHERE gt > 0)
  AND NOT EXISTS (SELECT 1 FROM UNNEST(call.genotype) AS gt WHERE gt < 0)
GROUP BY
  call_name
ORDER BY
  call_name

,call_name,call_count_for_call_set
0,NA12877,4486610
1,NA12878,4502017
2,NA12889,4422706
3,NA12890,4528725
4,NA12891,4424094
5,NA12892,4495753


In [0]:
%%bigquery --project cool-eye-258712
SELECT
  COUNT(DISTINCT call.name) AS number_of_callsets
FROM
  `bigquery-public-data.human_genome_variants.platinum_genomes_deepvariant_variants_20180823` v,  v.call

,number_of_callsets
0,6


In [0]:
%%bigquery --project cool-eye-258712
SELECT
  call_filter,
  COUNT(call_filter) AS number_of_calls
FROM
  `bigquery-public-data.human_genome_variants.platinum_genomes_deepvariant_variants_20180823` v,
  v.call,
  UNNEST(call.FILTER) AS call_filter
GROUP BY
  call_filter
ORDER BY
  number_of_calls

,call_filter,number_of_calls
0,RefCall,11681534
1,PASS,26867854


In [0]:
%%bigquery --project cool-eye-258712
SELECT
  reference_name,
  start_position,
  end_position,
  reference_bases,
  call.name AS call_name,
  (SELECT STRING_AGG(call_filter) FROM UNNEST(call.FILTER) AS call_filter) AS filters,
  ARRAY_LENGTH(call.FILTER) AS filter_count
FROM
  `bigquery-public-data.human_genome_variants.platinum_genomes_deepvariant_variants_20180823` v, v.call
WHERE
  EXISTS (SELECT 1 FROM UNNEST(call.FILTER) AS call_filter WHERE call_filter = 'PASS')
  AND ARRAY_LENGTH(call.FILTER) > 1
ORDER BY
  filter_count DESC, reference_name, start_position, end_position, reference_bases, call_name
LIMIT
  10

,reference_name,start_position,end_position,reference_bases,call_name,filters,filter_count


In [0]:
%%bigquery --project cool-eye-258712
SELECT
  call.name AS call_name,
  COUNT(1) AS number_of_calls
FROM
  `bigquery-public-data.human_genome_variants.platinum_genomes_deepvariant_variants_20180823` v, v.call
WHERE
  NOT EXISTS (SELECT 1 FROM UNNEST(call.FILTER) AS call_filter WHERE call_filter != 'PASS')
GROUP BY
  call_name
ORDER BY
  call_name

,call_name,number_of_calls
0,NA12877,29795946
1,NA12878,26118774
2,NA12889,29044992
3,NA12890,28717437
4,NA12891,31395995
5,NA12892,25349974


In [0]:
%%bigquery --project cool-eye-258712
SELECT
  call.name AS call_name,
  COUNT(1) AS number_of_calls
FROM
  `bigquery-public-data.human_genome_variants.platinum_genomes_deepvariant_variants_20180823` v, v.call
WHERE
  NOT EXISTS (SELECT 1 FROM UNNEST(call.FILTER) AS call_filter WHERE call_filter != 'PASS')
  AND EXISTS (SELECT 1 FROM UNNEST(call.genotype) as gt WHERE gt > 0)
GROUP BY
  call_name
ORDER BY
  call_name

,call_name,number_of_calls
0,NA12877,4486610
1,NA12878,4502017
2,NA12889,4422706
3,NA12890,4528725
4,NA12891,4424094
5,NA12892,4495753


In [0]:
%%bigquery --project cool-eye-258712
SELECT
  REGEXP_REPLACE(reference_name, '^chr', '') AS chromosome,
  COUNT(reference_name) AS number_of_variant_rows
FROM
  `bigquery-public-data.human_genome_variants.platinum_genomes_deepvariant_variants_20180823` v
WHERE
  EXISTS (SELECT 1
            FROM UNNEST(v.call) AS call, UNNEST(call.genotype) AS gt
          WHERE gt > 0)
GROUP BY
  chromosome
ORDER BY
  chromosome

,chromosome,number_of_variant_rows
0,1,615000
1,10,396773
2,11,391260
3,12,382841
4,13,298044
5,14,258756
6,15,234569
7,16,247671
8,17,224403
9,18,227200


In [0]:
%%bigquery --project cool-eye-258712
SELECT
  reference_name,
  COUNT(reference_name) AS number_of_variant_rows
FROM
  `bigquery-public-data.human_genome_variants.platinum_genomes_deepvariant_variants_20180823` v
WHERE
  EXISTS (SELECT 1
            FROM UNNEST(v.call) AS call
          WHERE EXISTS (SELECT 1
                          FROM UNNEST(call.genotype) AS gt
                        WHERE gt > 0))
GROUP BY
  reference_name
ORDER BY
  reference_name

,reference_name,number_of_variant_rows
0,chr1,615000
1,chr10,396773
2,chr11,391260
3,chr12,382841
4,chr13,298044
5,chr14,258756
6,chr15,234569
7,chr16,247671
8,chr17,224403
9,chr18,227200


Data Stories Start Below


In [20]:

QUERY = """ 
SELECT
  INTEGER(reference_name) AS chromosome,
  vt AS variant_type,
  COUNT(1) AS cnt
 FROM
   [genomics-public-data:1000_genomes.variants]
 OMIT RECORD IF
   reference_name IN ("X", "Y", "MT")
 GROUP BY
   chromosome,
   variant_type
        """

bq_assistant = BigQueryHelper("bigquery-public-data", "human_genome_variants")
bq_assistant.estimate_query_size(QUERY)

0.31291691306978464

In [0]:
%%bigquery --project cool-eye-258712

# Count the number of samples in the phenotypic data
SELECT
  COUNT(sample) AS all_samples,
  SUM(IF(In_Phase1_Integrated_Variant_Set = TRUE, 1, 0)) AS samples_in_variants_table
FROM
  `bigquery-public-data.human_genome_variants.1000_genomes_sample_info`

In [0]:
%%bigquery --project cool-eye-258712
SELECT
  gender,
  gender_count,
  gender_count / SUM(gender_count)
OVER
  (
  ORDER BY
    gender_count) AS gender_ratio
FROM (
  SELECT
    gender,
    COUNT(gender) AS gender_count
  FROM
    `bigquery-public-data.human_genome_variants.1000_genomes_sample_info`
  WHERE
    In_Phase1_Integrated_Variant_Set = TRUE
  GROUP BY
    gender)

In [13]:
%%bigquery --project cool-eye-258712 --use_legacy_sql
#legacySQL

SELECT
  population,
  population_description,
  population_count,
  RATIO_TO_REPORT(population_count)
OVER
  (
  ORDER BY
    population_count) AS population_ratio,
  super_population,
  super_population_description,
from(
  SELECT
    population,
    population_description,
    super_population,
    super_population_description,
    COUNT(population) AS population_count,
  FROM
    [genomics-public-data:1000_genomes.sample_info]
  WHERE
    In_Phase1_Integrated_Variant_Set = TRUE
  GROUP BY
    population,
    population_description,
    super_population,
    super_population_description)

,population,population_description,population_count,population_ratio,super_population,super_population_description
0,IBS,Iberian populations in Spain,14,0.012821,EUR,European
1,PUR,Puerto Rican in Puerto Rico,55,0.050366,AMR,American
2,CLM,"Colombian in Medellin, Colombia",60,0.054945,AMR,American
3,ASW,African Ancestry in Southwest US,61,0.055861,AFR,African
4,MXL,"Mexican Ancestry in Los Angeles, California",66,0.060440,AMR,American
5,CEU,Utah residents with Northern and Western Europ...,85,0.077839,EUR,European
6,YRI,"Yoruba in Ibadan, Nigeria",88,0.080586,AFR,African
7,GBR,British in England and Scotland,89,0.081502,EUR,European
8,JPT,"Japanese in Tokyo, Japan",89,0.081502,EAS,East Asian
9,FIN,Finnish in Finland,93,0.085165,EUR,European


In [15]:
%%bigquery --project cool-eye-258712 --use_legacy_sql
#legacySQL

# Ratios of ethnicities grouped by gender
SELECT
  population,
  gender,
  population_count,
  RATIO_TO_REPORT(population_count) OVER(
  PARTITION BY
    population
  ORDER BY
    gender)
  AS population_ratio
from(
  SELECT
    gender,
    population,
    COUNT(population) AS population_count,
  FROM
    [genomics-public-data:1000_genomes.sample_info]
  WHERE
    In_Phase1_Integrated_Variant_Set = TRUE
  GROUP BY
    gender,
    population)
ORDER BY
  population,
  gender

,population,gender,population_count,population_ratio
0,ASW,female,37,0.606557
1,ASW,male,24,0.393443
2,CEU,female,40,0.470588
3,CEU,male,45,0.529412
4,CHB,female,53,0.546392
5,CHB,male,44,0.453608
6,CHS,female,50,0.500000
7,CHS,male,50,0.500000
8,CLM,female,31,0.516667
9,CLM,male,29,0.483333


In [17]:
%%bigquery --project cool-eye-258712 --use_legacy_sql
#legacySQL

# Compute the distribution of family sizes
SELECT
num_family_members AS family_size,
COUNT(num_family_members) AS num_families_of_size
FROM (
  SELECT
  family_id,
  COUNT(family_id) AS num_family_members,
  FROM
  [genomics-public-data:1000_genomes.sample_info]
  WHERE
  In_Phase1_Integrated_Variant_Set = TRUE
  GROUP BY
  family_id)
GROUP BY
family_size

,family_size,num_families_of_size
0,1,636
1,2,204
2,3,12
3,4,3


In [18]:
%%bigquery --project cool-eye-258712 --use_legacy_sql
SELECT
   INTEGER(reference_name) AS chromosome,
   MIN(start) AS min,
   MAX(start) AS max
 FROM
   [genomics-public-data:1000_genomes.variants]
 OMIT RECORD IF
   reference_name IN ("X", "Y", "MT")
 GROUP BY
   chromosome

,chromosome,min,max
0,17,55,81194906
1,8,10421,146303865
2,14,19002083,107289452
3,15,20001199,102520964
4,5,11939,180885154
5,9,10022,141144794
6,6,73923,171051268
7,4,10239,191043592
8,11,70854,134946450
9,7,16160,159128573


In [22]:
%%bigquery --project cool-eye-258712 --use_legacy_sql

SELECT
  INTEGER(reference_name) AS chromosome,
  vt AS variant_type,
  COUNT(1) AS cnt
 FROM
   [genomics-public-data:1000_genomes.variants]
 OMIT RECORD IF
   reference_name IN ("X", "Y", "MT")
 GROUP BY
   chromosome,
   variant_type

,chromosome,variant_type,cnt
0,17,INDEL,39166
1,20,SV,342
2,15,INDEL,41178
3,18,SV,395
4,11,SV,724
...,...,...,...
61,8,INDEL,71739
62,22,SNP,475371
63,8,SV,732
64,18,SNP,1047971
